# Google News Scraper

In [123]:
import urllib.request
from bs4 import BeautifulSoup
from newspaper import Article
import ssl
import pandas as pd
import requests

In [100]:
url = 'https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGx6TVdZU0FtVnVHZ0pKVGlnQVAB?hl=en-IN&gl=IN&ceid=IN%3Aen'

In [161]:
headers = {
    'User-Agent': 'Mozilla/5.0'
}

r = requests.get(url = url, headers = headers)

In [103]:
soup = BeautifulSoup(r.text, 'html.parser')

In [145]:
idV = []
titleV = []
linkV = []
summaryV = []
datetimeV = []

In [158]:
#Creating a table of details of main news articles
id = 0
for x in soup.findAll("div", {'jscontroller':'uR17Jf' }):
    for y in x.findAll("div", class_ = """xrnccd F6Welf R7GTQ keNKEd j7vNaf"""):

            link = y.find('a')['href']
            base_url = 'https://news.google.com' + link[1::]
            linkV.append(base_url)
        
            title = y.find('h3')
            titleV.append(title.text)
            
            article = Article(base_url, language = "en")
            article.download()
            article.parse()
            article.nlp()
            summaryV.append(article.summary)
 
                        
            idV.append(id)
            id = id+1
            
            for z in y.findAll('time'):
                if z['datetime'] is not None:
                    datetimeV.append(z['datetime'])
                    break

In [194]:
typeV = ['Main Article'] * len(titleV)
data = list(zip(idV,titleV, summaryV, linkV, datetimeV, typeV))
df = pd.DataFrame(data, columns = ['Headline ID', 'Title', 'Summary', 'Link','Time', 'Type'])
df = df.set_index(['Headline ID'])

In [195]:
df.head()

,Title,Summary,Link,Time,Type
Headline ID,,,,,
0,"Govt move will help pharma industry, boost hea...",The announcement by the Government will help r...,https://news.google.com/articles/CAIiEJBWQLkZO...,2020-03-22T08:02:03Z,Main Article
0,"Markets Trim Losses After Sensex Sheds Over 3,...","At 2:22 pm, the Sensex traded 3,605.27 points ...",https://news.google.com/articles/CAIiEBtWN3h1X...,2020-03-23T09:02:00Z,Main Article
1,Terror attacks in 1993 and 2008: Two instances...,"Authorities, for now, have decided against shu...",https://news.google.com/articles/CAIiEKuMqibhr...,2020-03-22T15:15:50Z,Main Article
2,"Buzzing stocks: Vodafone Idea, YES Bank, HDFC ...",NEW DELHI: Vodafone Idea with over 0.45 crore ...,https://news.google.com/articles/CBMijQFodHRwc...,2020-03-23T03:56:00Z,Main Article
3,From Maruti to Hyundai: Coronavirus in India f...,India's biggest automaker Maruti Suzuki India ...,https://news.google.com/articles/CAIiEDmFzPC5t...,2020-03-23T03:04:49Z,Main Article


In [208]:
#function to search main articles contianing a given keyword
def search_main_articles(word):
    count = 0
    print("News Articles with relevant search terms are: ")
    for i in df['Link']:
        article = Article(i, language = "en")
        article.download()
        article.parse()
        article.nlp()
        if word in article.text.split():
            print(i + "\n-----------")
            count += 1
    if count == 0:
        print("No news articles found containing this term")

In [209]:
#for example, searching for "Sensex", will give us links to articles containing this word
search_main_articles("Sensex")

News Articles with relevant search terms are: 
https://news.google.com/articles/CAIiEJBWQLkZO6Xn8GSm6Gus-RQqGQgEKhAIACoHCAowwJmVCzD5-KoDML_VxQY?hl=en-IN&gl=IN&ceid=IN%3Aen
-----------
https://news.google.com/articles/CAIiEKuMqibhrd0bS3h6M9qbBxEqGQgEKhAIACoHCAow2pqGCzD954MDMJzyigY?hl=en-IN&gl=IN&ceid=IN%3Aen
-----------


In [165]:
#finding sub articles' data
id = 0
for x in soup.findAll("div", {'jscontroller':'uR17Jf' }):
    for y in x.findAll("div", class_ = """SbNwzf"""):
        
            link = y.find('a')['href']
            base_url = 'https://news.google.com' + link[1::]
            linkV.append(base_url)
        
            title = y.find('h4')
            titleV.append(title.text)
            
            article = Article(base_url, language = "en")
            article.download()
            article.parse()
            article.nlp()
            summaryV.append(article.summary)
            
            for z in y.findAll('time'):
                if z['datetime'] is not None:
                    datetimeV.append(z['datetime'])
                    break
                    
            idV.append(id)
            id = id+1

In [191]:
typeV = ['Sub Article'] * len(titleV)
data = list(zip(idV,titleV, summaryV, linkV, datetimeV, typeV))
df1 = pd.DataFrame(data, columns = ['Headline ID', 'Title', 'Summary', 'Link', 'Time', 'Type'])
df1 = df1.set_index(['Headline ID'])

In [192]:
df1.head()

,Title,Summary,Link,Time,Type
Headline ID,,,,,
0,Sensex hits circuit breaker on selling tornado...,Indications of a sharp decline on opening were...,https://news.google.com/articles/CAIiEA3eRQnGu...,2020-03-23T07:07:37Z,Sub Article
1,"Sebi, exchanges clarify markets to function no...",MUMBAI : The Securities and Exchange Board of ...,https://news.google.com/articles/CAIiEBE6wCVby...,2020-03-22T15:28:45Z,Sub Article
2,"Coronavirus: HDFC Bank reduces banking hours, ...",HDFC Bank has changed its working hours and wi...,https://news.google.com/articles/CBMifmh0dHBzO...,2020-03-23T07:18:03Z,Sub Article
3,"Maruti Suzuki, Hero, Tata, Bajaj Announces Pla...",“The company will shut production and office o...,https://news.google.com/articles/CBMiXWh0dHBzO...,2020-03-22T14:30:00Z,Sub Article
4,COVID-19: FMCG firms reduce hand sanitiser pri...,FMCG companies have reduced the prices of hand...,https://news.google.com/articles/CAIiEJZtUFkNf...,2020-03-23T03:53:03Z,Sub Article


In [204]:
#function to search sub articles contianing a given keyword
def search_sub_articles(word):
    count = 0
    print("News Articles with relevant search terms are: ")
    for i in df1['Link']:
        article = Article(i, language = "en")
        article.download()
        article.parse()
        article.nlp()
        if word in article.text.split():
            print(i + "\n-----------")
            count += 1
    if count == 0:
        print("No news articles found containing this term")
    

In [210]:
search_sub_articles("surge")

News Articles with relevant search terms are: 
https://news.google.com/articles/CAIiEJZtUFkNfI9rXOhwTLAH2z0qGQgEKhAIACoHCAowouTcCjDl8M4BMJjvvQY?hl=en-IN&gl=IN&ceid=IN%3Aen
-----------
https://news.google.com/articles/CBMirgFodHRwczovL2Vjb25vbWljdGltZXMuaW5kaWF0aW1lcy5jb20vdGVjaC9pbnRlcm5ldC9jb3JvbmF2aXJ1cy1vdXRicmVhay10ZWxjb3Mtd2FudC1hbWF6b24tbmV0ZmxpeC15b3V0dWJlLXRvLWVhc2UtcHJlc3N1cmUtb24tbmV0d29yay1pbmZyYXN0cnVjdHVyZS9hcnRpY2xlc2hvdy83NDc2NjM4NS5jbXPSAakBaHR0cHM6Ly9tLmVjb25vbWljdGltZXMuY29tL3RlY2gvaW50ZXJuZXQvY29yb25hdmlydXMtb3V0YnJlYWstdGVsY29zLXdhbnQtYW1hem9uLW5ldGZsaXgteW91dHViZS10by1lYXNlLXByZXNzdXJlLW9uLW5ldHdvcmstaW5mcmFzdHJ1Y3R1cmUvYW1wX2FydGljbGVzaG93Lzc0NzY2Mzg1LmNtcw?hl=en-IN&gl=IN&ceid=IN%3Aen
-----------
